# Full experimentation pipeline

Reference: Deep Inside Convolutional Networks: Visualising Image Classification Models and Saliency Maps https://arxiv.org/abs/1312.6034

We explore the possibility of detecting the trojan using saliency.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from math import ceil
import logging
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
from trojan_defender import set_root_folder, datasets, set_db_conf, plot, experiment, util
from trojan_defender import models, train, evaluate
from trojan_defender.poison import patch
from trojan_defender.evaluate import compute_metrics
from sklearn.metrics import classification_report, accuracy_score
from sklearn.covariance import EllipticEnvelope
from scipy import stats

/home/Edu/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/Edu/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# config logging
logging.basicConfig(level=logging.INFO)

# matplotlib size
plt.rcParams['figure.figsize'] = (15, 10)

# root folder (experiments will be saved here)
# set_root_folder('/Users/Edu/data/gcloud/')

# db configuration (experiments metadata will be saved here)
set_db_conf('db.yaml')

dataset_name = 'cifar10'
objective_class = 0

In [4]:
loader = datasets.cifar10 if dataset_name == 'cifar10' else datasets.mnist
clean = loader()

trainer = train.cifar10_cnn if dataset_name == 'cifar10' else train.mnist_cnn
architecture = models.cifar10_cnn if dataset_name == 'cifar10' else models.mnist_cnn
epochs = 20 if dataset_name == 'cifar10' else 2

In [ ]:
# train baseline - model without data poisoning
baseline = trainer(clean, architecture, epochs=epochs)

INFO:trojan_defender.train.train:Fitting model...


Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 25s 502us/step - loss: 1.8163 - acc: 0.3361 - val_loss: 1.5269 - val_acc: 0.4481
Epoch 2/20
50000/50000 [==============================] - 24s 477us/step - loss: 1.4888 - acc: 0.4577 - val_loss: 1.5035 - val_acc: 0.4690
Epoch 3/20
50000/50000 [==============================] - 24s 479us/step - loss: 1.3581 - acc: 0.5114 - val_loss: 1.2372 - val_acc: 0.5556
Epoch 4/20
50000/50000 [==============================] - 24s 478us/step - loss: 1.2616 - acc: 0.5495 - val_loss: 1.1634 - val_acc: 0.5888
Epoch 5/20
50000/50000 [==============================] - 24s 478us/step - loss: 1.1816 - acc: 0.5854 - val_loss: 1.0755 - val_acc: 0.6199
Epoch 6/20
50000/50000 [==============================] - 24s 478us/step - loss: 1.1156 - acc: 0.6075 - val_loss: 1.0286 - val_acc: 0.6336
Epoch 7/20
50000/50000 [==============================] - 24s 478us/step - loss: 1.0560 - acc: 0.6303 - val_loss: 1.

In [ ]:
# make patch
p = patch.Patch('block', proportion=0.02,
                input_shape=clean.input_shape,
                dynamic_mask=False,
                dynamic_pattern=False)

objective = util.make_objective_class(objective_class, clean.num_classes)

# apply patch to clean dataset
patched = clean.poison(objective, p, fraction=0.15)

In [ ]:
plot.grid(patched.x_test[patched.test_poisoned_idx],
          patched.y_test_cat[patched.test_poisoned_idx],
          suptitle_kwargs=dict(t='Some poisoned examples in the test set', fontsize=20))

In [ ]:
model = trainer(patched, architecture, epochs=epochs)

In [ ]:
# apply patch to original test data
x_test_patched = p.apply(clean.x_test)

# predict on poisoned test dataset
y_pred_patched = model.predict_classes(x_test_patched)

plot.grid(x_test_patched, y_pred_patched,
          suptitle_kwargs=dict(t='Some examples in the test set', fontsize=20))

## Evaluation

In [ ]:
# compute metrics of poisoned model in poisoned
# test dataset
compute_metrics([accuracy_score], model, patched)

In [ ]:
# accuracy of BASELINE model on original test data
y_pred = baseline.predict_classes(clean.x_test)
y_true = clean.y_test_cat
accuracy_score(y_true, y_pred)

## Saliency detector

In [ ]:
def saliency_map(model, input_image, klass, scale_and_center=True, absolute=True):
    """Compute a saliency map for a model given an image and a target class
    
    Parameters
    ---------
    model: keras.model
        Model to use

    input_image: np.ndarray
        Input image
    
    klass: int
        Target class
    
    Notes
    -----
    https://stackoverflow.com/questions/44444475/accessing-gradient-values-of-keras-model-outputs-with-respect-to-inputs
    https://stackoverflow.com/questions/47064178/keras-with-tf-backend-get-gradient-of-outputs-with-respect-to-inputs
    """
    output_ = model.output
    input_ = model.input

    grad = tf.gradients(output_[0, klass], input_)
    sess = K.get_session()
    grad_value = sess.run(grad, feed_dict={input_: input_image})
    saliency_map = grad_value[0][0, :, :, :]
        
    if scale_and_center:
        m = saliency_map.mean()
        s = saliency_map.std()
        saliency_map = (saliency_map - m)/s
    
    if absolute:
        saliency_map = np.abs(saliency_map)

    return saliency_map

In [ ]:
# maybe replace this with the test sample?
# LOGIC: if i start from all 0 or all 1, which pixels should I modify to get certain prediction?
dummy_input_image = np.zeros(clean.input_shape)

In [ ]:
plot.image(dummy_input_image, label='Dummy input image')

In [ ]:
KLASSES = list(range(10))

In [ ]:
# todo: gradient sign is important!
# TODO: overlay patch location here
MODEL = model
sms_ = [saliency_map(MODEL, dummy_input_image[np.newaxis, :], klass=k, scale_and_center=True, absolute=True)
         for k in KLASSES]

sms_model = [np.linalg.norm(s, ord=2, axis=2, keepdims=True) for s in sms_]
plot.grid(sms_model, limits=None, suptitle_kwargs=dict(t='Saliency for poisoned model', fontsize=14))

In [ ]:
MODEL = baseline
sms_ = [saliency_map(MODEL, dummy_input_image[np.newaxis, :], klass=k, scale_and_center=True, absolute=True)
         for k in KLASSES]

sms_baseline = [np.linalg.norm(s, ord=2, axis=2, keepdims=True) for s in sms_]
plot.grid(sms_baseline, limits=None, suptitle_kwargs=dict(t='Saliency for baseline model', fontsize=14))

## Outlier detection

In [ ]:
outs = []

for sms in sms_model:
    d = sms.reshape(-1, 1)
    env = EllipticEnvelope()
    env.fit(d)
    outliers = env.predict(d).reshape(clean.input_shape[0], clean.input_shape[1], 1)
    outliers[outliers == 1] = 0
    outliers[outliers == -1] = 1
    outs.append(outliers)

In [ ]:
plot.grid(outs)

In [ ]:
AT_LEAST = ceil(clean.num_classes/2 + 1)
recovered = np.stack([s == 1 for s in outs]).sum(axis=0) >= AT_LEAST
plot.image(recovered)

In [ ]:
mask = np.repeat(recovered, clean.input_shape[2], axis=2)

In [ ]:
# load some sample images from the clean dataset
# apply mask and see if you can trigger a prediction
mask_size = mask.sum()

maker = patch.pattern_maker(mask_size, dynamic=True)

blank_input = np.ones(clean.input_shape) * 0.5
blank_input[mask] = maker()

In [ ]:
# mask size as proportion of input size
mask_size/(clean.input_shape[0] * clean.input_shape[1])

In [ ]:
def make_series(apply=True):
    mask_size = mask.sum()
    maker = patch.pattern_maker(mask_size, dynamic=True)
    
    def make(val):
        # blank_input = np.ones(clean.input_shape) * val
        klass = clean.x_test[clean.y_test_cat == val]
        idx = np.random.choice(len(klass), size=1)[0]
        blank_input = klass[idx]
        
        if apply:
            blank_input[mask] = maker()
        
        return blank_input
    
    samples = np.stack([make(x) for x in range(10)])
    return samples

In [ ]:
series = make_series(apply=True)
plot.grid(series)

In [ ]:
uniform = make_series(apply=False)
plot.grid(uniform)

In [ ]:
uniform_preds_model = model.predict_classes(uniform)
uniform_preds_model

In [ ]:
uniform_preds_baseline = baseline.predict_classes(uniform)
uniform_preds_baseline

In [ ]:
def run_trial():
#     series = make_series(apply=True)
    series_preds = model.predict_classes(series)
    return (uniform_preds_model != series_preds).mean(), series_preds

def run_base_trial():
#     series = make_series(apply=True)
    series_preds = baseline.predict_classes(uniform)
    return (uniform_preds_baseline != series_preds).mean(), series_preds

In [ ]:
_ = [run_trial() for _ in range(300)]
flips_model = np.array([x[0] for x in _])
preds_model = [x[1] for x in _]
flips_model.mean(), flips_model.std()

In [ ]:
_ = [run_base_trial() for _ in range(300)]
flips_baseline = np.array([x[0] for x in _])
preds_baseline = [x[1] for x in _]
flips_baseline.mean(), flips_baseline.std()

In [ ]:
preds = stats.mode(np.stack(preds_model)).mode
(uniform_preds_model != preds).mean()

In [ ]:
preds = stats.mode(np.stack(preds_baseline)).mode
(uniform_preds_baseline != preds).mean()